# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 15 2023 9:14AM,264796,16,SHL-9573033,"SHL Pharma, LLC",Released
1,Jun 15 2023 9:12AM,264798,10,EYE43102,Eye Pharma Inc,Released
2,Jun 15 2023 9:12AM,264798,10,EYE43103,Eye Pharma Inc,Released
3,Jun 15 2023 9:11AM,264797,10,Enova-11968,Emerginnova,Released
4,Jun 15 2023 9:11AM,264797,10,Enova-11969,Emerginnova,Released
5,Jun 15 2023 9:11AM,264797,10,Enova-11970,Emerginnova,Released
6,Jun 15 2023 8:58AM,264795,20,9574628,"Exact-Rx, Inc.",Released
7,Jun 15 2023 8:58AM,264795,20,9574629,"Exact-Rx, Inc.",Released
8,Jun 15 2023 8:53AM,264794,10,HH-42603,Hush Hush,Released
9,Jun 15 2023 8:53AM,264794,10,HH-42604,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,264794,Released,17
23,264795,Released,2
24,264796,Released,1
25,264797,Released,3
26,264798,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264794,NaN,17.0
264795,NaN,2.0
264796,NaN,1.0
264797,NaN,3.0
264798,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264726,2.0,1.0
264733,11.0,2.0
264740,0.0,1.0
264746,0.0,1.0
264757,2.0,22.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264726,2,1
264733,11,2
264740,0,1
264746,0,1
264757,2,22


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264726,2,1
1,264733,11,2
2,264740,0,1
3,264746,0,1
4,264757,2,22


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264726,2,1
1,264733,11,2
2,264740,,1
3,264746,,1
4,264757,2,22


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 15 2023 9:14AM,264796,16,"SHL Pharma, LLC"
1,Jun 15 2023 9:12AM,264798,10,Eye Pharma Inc
3,Jun 15 2023 9:11AM,264797,10,Emerginnova
6,Jun 15 2023 8:58AM,264795,20,"Exact-Rx, Inc."
8,Jun 15 2023 8:53AM,264794,10,Hush Hush
25,Jun 15 2023 8:52AM,264793,10,Alikai Health
42,Jun 15 2023 8:49AM,264792,10,"CLINUVEL, Inc."
44,Jun 15 2023 8:45AM,264790,10,"Zelenko Labs, LLC"
45,Jun 15 2023 8:39AM,264789,10,"Snap Medical Industries, LLC"
48,Jun 15 2023 8:37AM,264788,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 15 2023 9:14AM,264796,16,"SHL Pharma, LLC",,1
1,Jun 15 2023 9:12AM,264798,10,Eye Pharma Inc,,2
2,Jun 15 2023 9:11AM,264797,10,Emerginnova,,3
3,Jun 15 2023 8:58AM,264795,20,"Exact-Rx, Inc.",,2
4,Jun 15 2023 8:53AM,264794,10,Hush Hush,,17
5,Jun 15 2023 8:52AM,264793,10,Alikai Health,,17
6,Jun 15 2023 8:49AM,264792,10,"CLINUVEL, Inc.",,2
7,Jun 15 2023 8:45AM,264790,10,"Zelenko Labs, LLC",,1
8,Jun 15 2023 8:39AM,264789,10,"Snap Medical Industries, LLC",,3
9,Jun 15 2023 8:37AM,264788,10,"Snap Medical Industries, LLC",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 15 2023 9:14AM,264796,16,"SHL Pharma, LLC",1,
1,Jun 15 2023 9:12AM,264798,10,Eye Pharma Inc,2,
2,Jun 15 2023 9:11AM,264797,10,Emerginnova,3,
3,Jun 15 2023 8:58AM,264795,20,"Exact-Rx, Inc.",2,
4,Jun 15 2023 8:53AM,264794,10,Hush Hush,17,
5,Jun 15 2023 8:52AM,264793,10,Alikai Health,17,
6,Jun 15 2023 8:49AM,264792,10,"CLINUVEL, Inc.",2,
7,Jun 15 2023 8:45AM,264790,10,"Zelenko Labs, LLC",1,
8,Jun 15 2023 8:39AM,264789,10,"Snap Medical Industries, LLC",3,
9,Jun 15 2023 8:37AM,264788,10,"Snap Medical Industries, LLC",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 15 2023 9:14AM,264796,16,"SHL Pharma, LLC",1,
1,Jun 15 2023 9:12AM,264798,10,Eye Pharma Inc,2,
2,Jun 15 2023 9:11AM,264797,10,Emerginnova,3,
3,Jun 15 2023 8:58AM,264795,20,"Exact-Rx, Inc.",2,
4,Jun 15 2023 8:53AM,264794,10,Hush Hush,17,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 15 2023 9:14AM,264796,16,"SHL Pharma, LLC",1.0,NaN
1,Jun 15 2023 9:12AM,264798,10,Eye Pharma Inc,2.0,NaN
2,Jun 15 2023 9:11AM,264797,10,Emerginnova,3.0,NaN
3,Jun 15 2023 8:58AM,264795,20,"Exact-Rx, Inc.",2.0,NaN
4,Jun 15 2023 8:53AM,264794,10,Hush Hush,17.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 15 2023 9:14AM,264796,16,"SHL Pharma, LLC",1.0,0.0
1,Jun 15 2023 9:12AM,264798,10,Eye Pharma Inc,2.0,0.0
2,Jun 15 2023 9:11AM,264797,10,Emerginnova,3.0,0.0
3,Jun 15 2023 8:58AM,264795,20,"Exact-Rx, Inc.",2.0,0.0
4,Jun 15 2023 8:53AM,264794,10,Hush Hush,17.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3177415,52.0,12.0
12,264777,0.0,1.0
16,264796,1.0,0.0
19,1588652,37.0,0.0
20,794278,25.0,4.0
21,264746,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3177415,52.0,12.0
1,12,264777,0.0,1.0
2,16,264796,1.0,0.0
3,19,1588652,37.0,0.0
4,20,794278,25.0,4.0
5,21,264746,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,52.0,12.0
1,12,0.0,1.0
2,16,1.0,0.0
3,19,37.0,0.0
4,20,25.0,4.0
5,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,12,Released,0.0
2,16,Released,1.0
3,19,Released,37.0
4,20,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,12.0,1.0,0.0,0.0,4.0,0.0
Released,52.0,0.0,1.0,37.0,25.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,12.0,1.0,0.0,0.0,4.0,0.0
1,Released,52.0,0.0,1.0,37.0,25.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,12.0,1.0,0.0,0.0,4.0,0.0
1,Released,52.0,0.0,1.0,37.0,25.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()